In [1]:
source = "Recommendations";

In [2]:
using NBInclude
@nbinclude("../ProductionAlphas/Alpha.ipynb");

In [3]:
ENV["COLUMNS"] = 999999;
#ENV["LINES"] = 100;

In [4]:
anime = DataFrame(CSV.File("../../data/raw_data/anime.csv"))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [5]:
predictions = read_recommendee_alpha("CombineSignals").rating
rating_df = DataFrame("uid" => 0:length(predictions)-1, "rating" => predictions);

In [6]:
# evaluate our insample predictions
df = get_recommendee_list()
@debug rmse(df.rating, predictions[df.item])
@debug mae(df.rating, predictions[df.item])
@debug r2(df.rating, predictions[df.item])

[ Debug: 20220109 01:05:40 1.323205342864471
[ Debug: 20220109 01:05:41 1.0072534355516645
[ Debug: 20220109 01:05:41 0.4037941932263426


In [7]:
# don't recommend shows that the user has already seen before
rating_df.rating[get_recommendee_list().item] .= 0
rating_df.rating[get_implicit_list().item] .= 0;

In [8]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
rec_df = rec_df[:, ["anime_id", "title", "genres", "medium", "rating"]];
filter(x -> x.medium == "tv", sort!(rec_df, :rating, rev = true))

,anime_id,title,genres,medium,rating
,Int64,String,String,String7,Float64
1,42587,Yuuki Yuuna wa Yuusha de Aru: Dai Mankai no Shou,"['Drama', 'Fantasy', 'Slice of Life']",tv,8.31332
2,2921,Ashita no Joe 2,"['Action', 'Drama', 'Shounen', 'Slice of Life', 'Sports']",tv,8.09726
3,45576,Mushoku Tensei: Isekai Ittara Honki Dasu Part 2,"['Drama', 'Ecchi', 'Fantasy']",tv,7.8638
4,38524,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Fantasy', 'Military', 'Mystery', 'Shounen', 'Super Power']",tv,7.73232
5,28891,Haikyuu!! Second Season,"['Comedy', 'Drama', 'School', 'Shounen', 'Sports']",tv,7.6985
6,32843,Senki Zesshou Symphogear XV,"['Action', 'Music', 'Sci-Fi']",tv,7.60041
7,32935,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou,"['Comedy', 'Drama', 'School', 'Shounen', 'Sports']",tv,7.54376
8,5114,Fullmetal Alchemist: Brotherhood,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Military', 'Shounen']",tv,7.47793
9,41169,Love Live! Superstar!!,"['Music', 'School', 'Slice of Life']",tv,7.45217


In [9]:
sort!(rec_df, :rating, rev = true)

,anime_id,title,genres,medium,rating
,Int64,String,String,String7,Float64
1,34376,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 3 - Yakusoku,"['Drama', 'Fantasy', 'Slice of Life']",movie,9.07414
2,11979,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari,"['Drama', 'Psychological', 'Suspense']",movie,8.9394
3,34375,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 2 - Tamashii,"['Drama', 'Fantasy', 'Slice of Life']",movie,8.92491
4,11977,Mahou Shoujo Madoka★Magica Movie 1: Hajimari no Monogatari,"['Drama', 'Psychological', 'Suspense']",movie,8.76122
5,34374,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 1 - Tomodachi,"['Drama', 'Fantasy', 'Slice of Life']",movie,8.50913
6,42587,Yuuki Yuuna wa Yuusha de Aru: Dai Mankai no Shou,"['Drama', 'Fantasy', 'Slice of Life']",tv,8.31332
7,44042,Holo no Graffiti,['Comedy'],ona,8.21184
8,2921,Ashita no Joe 2,"['Action', 'Drama', 'Shounen', 'Slice of Life', 'Sports']",tv,8.09726
9,45576,Mushoku Tensei: Isekai Ittara Honki Dasu Part 2,"['Drama', 'Ecchi', 'Fantasy']",tv,7.8638
